***
# Model Management of Python using SAS Viya
***

#### Contacts:
* Yi Jian Ching, SAS AP (yijian.ching@sas.com)
* Thierry Jossermoz, SAS NZ (thierry.jossermoz@sas.com)

This notebook is an example of how you can use the SAS Viya Model Manager API to bring in models from an external Python environment for model governance.

In [1]:
import sys
print(sys.version)

3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]


### Python Package for Model Management
***
This notebook makes use of a high-level Python library designed to make it simpler for Python users to create model projects & repositories, import models & manage and govern their models through the Model Manager API.

In [2]:
from viyapy import ViyaClient
from viyapy.variables import String, Decimal, Boolean, Date, DateTime, Integer

In [3]:
#Configure logging if required
import logging
logger = logging.getLogger('viyapy')
logger.addHandler(logging.StreamHandler(sys.stdout))
logger.setLevel(logging.INFO)

### Model Management
***
Pushing Python code to centralised model repository on SAS Viya using REST APIs:

* Authenticate with Model Manager
* Retrieve repository metadata (repositoryId)
* Create Model Project
* Import Model
* Import Model files
* Create Model Performance definition

### Authenticate client with Viya server

In [4]:
protocol = 'http'
server = 'YourViyaServer.com'
user = 'sasdemo'

import getpass
password = getpass.getpass()

········


In [5]:
client = ViyaClient(protocol + '://' + server)
token = client.logon.authenticate(user, password)

Token retrieved successfully (expires in: 43199)


### Retrieve data about Model Repository
Repository ID is required to create Model Project.

In [6]:
repositoryName = 'Repository 1'
repository = client.model_manager.get_repository(repositoryName)

Retrieved successfully: Repository <ID: '232347d9-02db-42bd-b8c5-032473675b4e', Name: 'Repository 1'>


### Create a Model Project

In [7]:
project = repository.create_project(
    name='MAP_Python',
    description='Project demonstrating management of open source models in Viya',
    function='Classification',
    target_level='binary',
    target_event_value='1',
    class_target_values='1,0',
    target_variable='BAD',
    event_probability_variable='P_BAD1',
    external_url='http://github.com/user/project',
    input=[
        Decimal('BAD', level='binary'),
        Decimal('CLAGE'),
        Decimal('CLNO'),
        Decimal('DEBTINC'),
        Decimal('DEROG'),
        String('JOB'),
        Decimal('LOAN'),
        Decimal('MORTDUE'),
        Decimal('NINQ'),
        String('REASON'),
        Decimal('VALUE'),
        Integer('YOJ')
    ],
    output=[
        Decimal('P_BAD0', description='Probability of not defaulting'),
        Decimal('P_BAD1', description='Probability of defaulting')
    ])

Created successfully: Project <ID: '5a91bcc9-0e74-4e04-b0ac-5bd15ec7797f', Name: 'MAP_Python'>
Updated successfully: Project <ID: '5a91bcc9-0e74-4e04-b0ac-5bd15ec7797f', Name: 'MAP_Python'>


The created Model Project or an existing one can be retrieved like so:
```{python}
project = client.model_manager.get_project('modelId')
```

### Import a Model into Project
A model with metadata is registered against the project. Relevant model files (training code, scoring code and PKL file) can also be imported and attached to the model.

In [8]:
model = project.create_model(
    name='Python_GradientBoost',
    algorithm='scikit-learn.GradientBoostingClassifier',
    modeler='Yi Jian Ching'#,
    #files=[
    #    os.getcwd() + '/Modern_Analytics_Platform/gboost_train.py',
    #    os.getcwd() + '/Modern_Analytics_Platform/gboost_score.py',
    #    (os.getcwd() +'/Modern_Analytics_Platform/gboost_obj_3_6_6.pkl', True)
    #]
)

Created successfully: Model <ID: '831e873d-3766-44ac-b2ee-2bd2951c49d8', Name: 'Python_GradientBoost'>


### Import Model content
Model content can also be imported and attached separately - this is if you want to return the contentId(s) as a Python variable.

In [9]:
content_list = model.add_files(
        '/home/sasdemo/data/gboost_train.py',
        '/home/sasdemo/data/gboost_score.py',
        ('/home/sasdemo/data/model/gboost_obj_3_6_5.pkl', True)
)

Created successfully: File <ID: 'c274f44f-4b38-47e0-a83f-a0c59dddbaf4', Name: 'gboost_train.py'>
Created successfully: File <ID: 'cb4189c8-65c9-411b-9be2-4beef3d7c065', Name: 'gboost_score.py'>
Created successfully: File <ID: '051df148-44ad-479c-9e81-906afe8e9ccc', Name: 'gboost_obj_3_6_5.pkl'>


#### Create Model Performance Definition
Create the definition of the Model Performance Monitoring task for future execution.

In [10]:
perf_def = project.create_performance_definition(
    name='Performance Monitoring',
    models=[model])

Created successfully: PerformanceDefinition <ID: 'ec90c7c9-4f50-4fcb-80b4-d23a41b5a35c', Name: 'Performance Monitoring'>
